В этом проекте применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные. 

In [60]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

**Загрузите данные, проверьте типы, пропущенные значения.**

Сколько пропущенных значений встретилось в датасете?

In [61]:
df_cars = pd.read_csv('cars.csv')

In [62]:
df_cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [63]:
df_cars.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [64]:
df_cars.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [65]:
df_cars.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [66]:
# 0 пропущенных значений

**Сгенерируем первый признак.**

**Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент.**

**Например:**
'audi 100 ls' → 'audi'

Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.

In [67]:
df_cars['car_brand'] = df_cars['CarName'].apply(lambda x: x.split(' ')[0])

In [68]:
df_cars = df_cars.drop(columns = ['car_ID', 'CarName'])

In [69]:
df_cars.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,car_brand
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [70]:
df_cars.car_brand.nunique()

28

**Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!**

- 'maxda' → 'mazda'
- 'Nissan' → 'nissan'
- 'porcshce' → 'porsche'
- 'toyouta' → 'toyota'
- 'vokswagen' & 'vw' → 'volkswagen'

Сколько уникальных производителей осталось в итоге?

In [71]:
df_cars.car_brand.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [72]:
df_cars.car_brand = df_cars.car_brand.str.lower()\
    .replace(['maxda', 'porcshce', 'toyouta', 'vokswagen', 'vw'], ['mazda', 'porsche', 'toyota', 'volkswagen', 'volkswagen'])

In [73]:
df_cars.car_brand.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [74]:
df_cars.car_brand.nunique()

22

**Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:**

**'company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio', 'horsepower'**

также не забыв про то, что мы предсказываем – 'price'. 

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.

In [75]:
df_cars_corr_test = df_cars[['car_brand', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio', 'horsepower', 'price']]

In [76]:
df_cars_corr_test_result = round(df_cars_corr_test.corr(), 2)

In [77]:
df_cars_corr_test_result # результаты теста на корреляцию

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.00,0.87,0.80,0.78,0.57,0.49,0.35,0.58
carlength,0.87,1.00,0.84,0.88,0.68,0.61,0.55,0.68
carwidth,0.80,0.84,1.00,0.87,0.74,0.56,0.64,0.76
curbweight,0.78,0.88,0.87,1.00,0.85,0.65,0.75,0.84
enginesize,0.57,0.68,0.74,0.85,1.00,0.58,0.81,0.87
boreratio,0.49,0.61,0.56,0.65,0.58,1.00,0.57,0.55
horsepower,0.35,0.55,0.64,0.75,0.81,0.57,1.00,0.81
price,0.58,0.68,0.76,0.84,0.87,0.55,0.81,1.00


**Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:**

df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)
Не забудьте присоединить к полученному датасету столбцы с переменными других типов :) Также обратите внимание: мы должны использовать тот же набор колонок, который был на предыдущем шаге!

Сколько колонок теперь имеется в датасете?

In [78]:
df_cars_corr_test.dtypes

car_brand          object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [79]:
df_dummy = pd.get_dummies(data=df_cars[[ 'car_brand', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype',
                                        'cylindernumber']], drop_first=True)

In [80]:
df_regress_test = pd.concat([df_dummy, df_cars_corr_test[['wheelbase', 'carlength', 'carwidth','curbweight',
                                                           'enginesize', 'boreratio', 'horsepower', 'price']]], axis=1)

In [81]:
df_regress_test.shape

(205, 49)

**Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.**

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [82]:
x = sm.add_constant(df_regress_test.horsepower) # добавляем константу
model = sm.OLS(df_regress_test.price, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Sun, 07 Aug 2022   Prob (F-statistic):           1.48e-48
Time:                        01:00:53   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3721.7615    929.849     -4.003      0.0

In [83]:
round(0.653 * 100)

65

**Теперь – две модели:**

- модель со всеми предикторами
- модель со всеми предикторами, кроме марок машин
- Обратите внимание на изменения в R2, коэффициентах и их значимости. Какую модель лучше оставить? 

**Модель со всеми предикторами**

In [84]:
x = df_regress_test.drop(labels='price', axis=1)
x = sm.add_constant(x)
model = sm.OLS(df_regress_test.price, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Sun, 07 Aug 2022   Prob (F-statistic):           4.86e-89
Time:                        01:00:54   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

**Модель со всеми предикторами, кроме марок машин**

In [87]:
df_regress_test_2 = df_regress_test\
    .drop(labels=['car_brand_audi', 'car_brand_bmw', 'car_brand_buick', 'car_brand_chevrolet',
                  'car_brand_dodge', 'car_brand_honda', 'car_brand_isuzu', 'car_brand_jaguar', 
                  'car_brand_mazda', 'car_brand_mercury', 'car_brand_mitsubishi', 
                  'car_brand_nissan', 'car_brand_peugeot', 'car_brand_plymouth', 
                  'car_brand_porsche', 'car_brand_renault', 'car_brand_saab', 'car_brand_subaru',
                  'car_brand_toyota', 'car_brand_volkswagen', 'car_brand_volvo'], axis=1)

x_2 = df_regress_test_2.drop(labels='price', axis=1)
x_2 = sm.add_constant(x_2)
model_2 = sm.OLS(df_regress_test_2.price, x_2).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Sun, 07 Aug 2022   Prob (F-statistic):           9.86e-81
Time:                        01:10:19   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

**Выбирвем вторую, ведь R2 изменился не очень сильно, а часть марок в первой модели вообще не значима** 

Выбранная модель **объясняет примерно 
90% дисперсии** (окр. до целого). **10 предикторов из 27 оказались не значимыми** (p > 0.05). Пример интерпретации: **при единичном изменении показателя horsepower, цена ВОЗРАСТАЕТ на 86.8164** (без округления).